## Purpose: Try different models-- Part2.
### Logistic Regression with scaling.

In [1]:
# import dependencies.
import pandas as pd
import numpy as np

from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from sklearn.model_selection import train_test_split

#### STEP1: Read in dataset.  Remove data from 2016-2019.
- data from 2016-2018 will be used to bs test the model.
- data from 2019 will be used to predict the winners of the 2019 WS.

In [2]:
# read in the data.
team_data = pd.read_csv("../../Resources/clean_data_1969.csv")
del team_data["Unnamed: 0"]
team_data.head()

,team,year,A,DP,E,G2,GS2,INN,PB,PO,...,SHO,SO1,SV,SVO,TBF,W,WHIP,WP,WPCT,winners
0,St. Louis Cardinals,2019,1033,114,43,104,936,8313.0,3,2771,...,4,895,33,42,3896,56,1.29,21,0.538,0
1,Arizona Diamondbacks,2019,1010,83,45,105,945,8538.0,2,2846,...,7,925,24,37,4001,53,1.28,35,0.505,0
2,Kansas City Royals,2019,990,105,45,106,954,8421.0,6,2807,...,5,816,24,41,4125,39,1.46,34,0.368,0
3,Houston Astros,2019,875,54,50,106,954,8589.0,6,2863,...,7,1074,27,42,3929,67,1.14,31,0.632,0
4,Tampa Bay Rays,2019,975,92,53,107,963,8760.0,11,2920,...,6,1037,26,43,3985,59,1.16,40,0.551,0


In [3]:
# remove data from 2016 through 2019.
team_data_new = team_data.loc[team_data["year"] < 2016]
team_data_new.head()

,team,year,A,DP,E,G2,GS2,INN,PB,PO,...,SHO,SO1,SV,SVO,TBF,W,WHIP,WP,WPCT,winners
120,San Francisco Giants,2015,1639,136,72,162,1458,13143.0,6,4381,...,11,1309,43,72,6048,87,1.21,40,0.537,0
121,Washington Nationals,2015,1425,142,73,162,1458,13137.0,17,4379,...,12,1476,46,60,6036,95,1.19,47,0.586,0
122,Houston Astros,2015,1599,135,77,162,1458,13212.0,18,4404,...,8,1396,44,64,6180,84,1.29,98,0.519,0
123,Detroit Tigers,2015,1537,148,75,161,1449,12852.0,5,4284,...,8,1232,47,66,6048,86,1.32,44,0.534,0
124,Boston Red Sox,2015,1427,139,75,162,1458,12957.0,37,4319,...,5,1362,43,61,6073,93,1.27,52,0.574,0


In [4]:
# models require numbers! Drop team and year columns.
target = team_data_new["winners"]
features = team_data_new.drop({"team", "year", "winners"}, axis=1)
feature_columns = list(features.columns)
print (target.shape)
print (features.shape)
print (feature_columns)

(1266,)
(1266, 59)
['A', 'DP', 'E', 'G2', 'GS2', 'INN', 'PB', 'PO', 'TC', '2B', '3B', 'AB', 'AO', 'BB', 'CS', 'G', 'GDP', 'H', 'HBP', 'HR', 'IBB', 'NP_x', 'OBP', 'OPS_x', 'PA', 'R', 'RBI', 'SAC', 'SB', 'SF', 'SLG', 'SO', 'TB', 'XBH', 'BB1', 'BK', 'CG', 'ER', 'ERA', 'G1', 'GF', 'GS', 'H1', 'HB', 'HR1', 'IBB1', 'IP', 'L', 'OBP1', 'R1', 'SHO', 'SO1', 'SV', 'SVO', 'TBF', 'W', 'WHIP', 'WP', 'WPCT']


In [5]:
# split data to train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

#### STEP2: Try Logistic Regression.

In [6]:
# reshape the target data.
target1 = target.values.reshape(-1, 1)

In [7]:
# split data into train and test sets.
X_train1, X_test1, y_train1, y_test1 = train_test_split(features, target1, random_state=42)

In [8]:
# create standard scaler model and fit the training data.
X_scaler = StandardScaler().fit(X_train1)
y_scaler = StandardScaler().fit(y_train1)

# transform the training and testing data.
X_train_scaled = X_scaler.transform(X_train1)
X_test_scaled = X_scaler.transform(X_test1)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [9]:
# fit the model.
model = LogisticRegression(solver="lbfgs")
model.fit(X_train_scaled, y_train1)

# predict.
prediction = model.predict(X_test_scaled)

# evaluate the model.
mse = mean_squared_error(y_test1, prediction)
r2 = model.score(X_test_scaled, y_test1)
print (f"MSE: {mse}")
print (f"R2: {r2}")

MSE: 0.03785488958990536
R2: 0.9621451104100947


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wow.  That's a pretty good fit!  Since the scores are so high, try bs test with 2016-2018 data. But wait...  

In [10]:
print (classification_report(y_test1, prediction, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       304
           1       1.00      0.08      0.14        13

   micro avg       0.96      0.96      0.96       317
   macro avg       0.98      0.54      0.56       317
weighted avg       0.96      0.96      0.95       317



Ah hah!.  The model is predicting 0's correctly but only predicts 1's 50% of the time.  And it isn't terribly accurate.  That means that the two classes (0, 1) are very imbalanced.  Need to address this one.

#### STEP3: Try Logistic Regression with Upsampled Data.

In [11]:
# start over.  grab the data.
team_data_new = team_data_new.reset_index().drop({"index"}, axis=1)
team_data_new.head()

,team,year,A,DP,E,G2,GS2,INN,PB,PO,...,SHO,SO1,SV,SVO,TBF,W,WHIP,WP,WPCT,winners
0,San Francisco Giants,2015,1639,136,72,162,1458,13143.0,6,4381,...,11,1309,43,72,6048,87,1.21,40,0.537,0
1,Washington Nationals,2015,1425,142,73,162,1458,13137.0,17,4379,...,12,1476,46,60,6036,95,1.19,47,0.586,0
2,Houston Astros,2015,1599,135,77,162,1458,13212.0,18,4404,...,8,1396,44,64,6180,84,1.29,98,0.519,0
3,Detroit Tigers,2015,1537,148,75,161,1449,12852.0,5,4284,...,8,1232,47,66,6048,86,1.32,44,0.534,0
4,Boston Red Sox,2015,1427,139,75,162,1458,12957.0,37,4319,...,5,1362,43,61,6073,93,1.27,52,0.574,0


In [12]:
# remove team and year.
feature_columns_new = feature_columns + ["winners"]
team_data_new = team_data[feature_columns_new]
team_data_new.head()

,A,DP,E,G2,GS2,INN,PB,PO,TC,2B,...,SHO,SO1,SV,SVO,TBF,W,WHIP,WP,WPCT,winners
0,1033,114,43,104,936,8313.0,3,2771,3847,157,...,4,895,33,42,3896,56,1.29,21,0.538,0
1,1010,83,45,105,945,8538.0,2,2846,3901,203,...,7,925,24,37,4001,53,1.28,35,0.505,0
2,990,105,45,106,954,8421.0,6,2807,3842,185,...,5,816,24,41,4125,39,1.46,34,0.368,0
3,875,54,50,106,954,8589.0,6,2863,3788,200,...,7,1074,27,42,3929,67,1.14,31,0.632,0
4,975,92,53,107,963,8760.0,11,2920,3948,195,...,6,1037,26,43,3985,59,1.16,40,0.551,0


In [13]:
# upsample for a more balanced dataset.
def upsample(dataset, no_samples):
    '''
    INPUT: 
    -dataset = dataset without team names and year.
    -n_samples = number of minority_unsampled.
    
    OUTPUT:
    -X_train_scaled = scaled X train data.
    -X_test_scaled = scaled X test data.
    -y_train = y train data
    -y_test = y test data
    
    DESCRIPTION:
    -dataset is taken in and split into minority and majority classes.
    -dataset is then upsampled for the mainority class
    -split the data into features and targets
    -split data into train and test sets
    -train and test sets were are scaled.
    '''
    
    # separate majority and minority classes.
    df_majority = dataset.loc[dataset["winners"] == 0]
    df_minority = dataset.loc[dataset["winners"] == 1]

    # upsample minority class.
    df_minority_unsampled = resample(df_minority,
                                    replace=True,
                                    n_samples=no_samples,
                                    random_state=123)

    # combine majority class with upsampled minority class.
    df_upsampled = pd.concat([df_majority, df_minority_unsampled])

    # separate features and target.
    y = df_upsampled["winners"]
    X = df_upsampled[feature_columns]
    
    # split into train and test sets.
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    # scale X_train and X_test.
    scaler = StandardScaler()

    # transform the training and testing data.
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

In [14]:
# Do three different upsamplings.
X_train_100, X_test_100, y_train_100, y_test_100 = upsample(team_data_new, 2234)
X_train_50, X_test_50, y_train_50, y_test_50 = upsample(team_data_new, 1117)
X_train_25, X_test_25, y_train_25, y_test_25 = upsample(team_data_new, 559)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anacon

In [15]:
def fit_logistic(X_train, X_test, y_train, y_test):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -X_test = scaled X test data.
    -y_train = y train data.
    -y_test = y test data.
    
    OUTPUT:
    -classification report (has F1 score, precision and recall).
    -model = saved model for prediction. 
    
    DESCRIPTION:
    -the scaled and split data is put through a logistic regression.
    -the model is trained.
    -a prediction is made.
    -print out the classification report and give the model.
    '''
    
    # fit the model.
    model = LogisticRegression(solver="lbfgs", max_iter= 2000)
    model.fit(X_train, y_train)

    # predict.
    prediction = model.predict(X_test)
    
    # print out the classification report
    print (classification_report(y_test, prediction, target_names=["0", "1"]))
    
    return model

In [16]:
# for 1 part 0 to 1 part 1
model_100 = fit_logistic(X_train_100, X_test_100, y_train_100, y_test_100)

              precision    recall  f1-score   support

           0       0.81      0.56      0.66       356
           1       0.76      0.91      0.83       537

   micro avg       0.77      0.77      0.77       893
   macro avg       0.78      0.73      0.74       893
weighted avg       0.78      0.77      0.76       893



In [17]:
# for 1 part 0 to 0.5 part 1
model_50 = fit_logistic(X_train_50, X_test_50, y_train_50, y_test_50)

              precision    recall  f1-score   support

           0       0.79      0.73      0.76       337
           1       0.70      0.76      0.73       277

   micro avg       0.74      0.74      0.74       614
   macro avg       0.74      0.75      0.74       614
weighted avg       0.75      0.74      0.74       614



In [18]:
# for 1 part 0 to 0.25 part 1
model_25 = fit_logistic(X_train_25, X_test_25, y_train_25, y_test_25)

              precision    recall  f1-score   support

           0       0.80      0.86      0.83       341
           1       0.55      0.44      0.49       133

   micro avg       0.74      0.74      0.74       474
   macro avg       0.67      0.65      0.66       474
weighted avg       0.73      0.74      0.73       474



Ok.  That looks a little better -- at least it is predicting 0 and 1s.  The model_100 seems to be most even between predicting 0 and 1s.  Use that one.

#### STEP4: Predict 2016-2018 winners.

In [19]:
def predict_the_winner(model, year, team_data, X_train):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -model = the saved model.
    -team_data = complete dataframe with all data.
    -year = the year want to look at.
    
    OUTPUT:
    -printed prediction.
    
    DESCRIPTION:
    -data from year of interest is isolated.
    -the data are scaled.
    -the prediction is made.
    -print out the resulting probability and the name of the team.
    '''
    
    # grab the data.
    team_data = team_data.loc[team_data["year"] == year].reset_index()
    
    # set features (no team, year, winners).
    # set target (winners).
    features = team_data[feature_columns]
    
    # scale X_train and X_test.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    features = scaler.fit_transform(features)
    
    # fit the model.
    probabilities = model.predict_proba(features)

    # convert predictions to datafram.e
    WS_predictions = pd.DataFrame(probabilities[:,1])

    # Sort the DataFrame (descending)
    WS_predictions = WS_predictions.sort_values(0, ascending=False)

    WS_predictions['Probability'] = WS_predictions[0]

    # Print 50 highest probability HoF inductees from still eligible players
    for i, row in WS_predictions.head(50).iterrows():
       prob = ' '.join(('WS Probability =', str(row['Probability'])))
       print('')
       print(prob)
       print(team_data.iloc[i, :]["team"])

##### 2018 Prediction

In [20]:
# predict for 2018.
predict_the_winner(model_100, 2018, team_data, X_train_100)


WS Probability = 0.9999591783394074
Houston Astros

WS Probability = 0.9997093626463864
Boston Red Sox

WS Probability = 0.9994985821099432
Tampa Bay Rays

WS Probability = 0.9975619655926621
Los Angeles Dodgers

WS Probability = 0.9969699185170013
Los Angeles Angels

WS Probability = 0.9967262055233389
Minnesota Twins

WS Probability = 0.9904126347226426
Cleveland Indians

WS Probability = 0.9746767579057358
Arizona Diamondbacks

WS Probability = 0.954376127437217
Milwaukee Brewers

WS Probability = 0.9534381324942981
Oakland Athletics

WS Probability = 0.9457511544689748
Atlanta Braves

WS Probability = 0.8723945918196689
Washington Nationals

WS Probability = 0.8581970495481278
St. Louis Cardinals

WS Probability = 0.5740309426241594
Texas Rangers

WS Probability = 0.5417861356965357
San Francisco Giants

WS Probability = 0.487575308098069
New York Yankees

WS Probability = 0.43999424503010426
Pittsburgh Pirates

WS Probability = 0.4070120756026965
New York Mets

WS Probability = 0

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


This does not look good.  The Red Sox won 2018.

##### 2017 Prediction

In [21]:
# predict for 2017.
predict_the_winner(model_100, 2017, team_data, X_train_100)


WS Probability = 0.9999929107238257
Cleveland Indians

WS Probability = 0.9999741983649034
Boston Red Sox

WS Probability = 0.9997981828996244
Houston Astros

WS Probability = 0.9997084045641971
Washington Nationals

WS Probability = 0.999630006196054
Los Angeles Dodgers

WS Probability = 0.9992428962606721
Atlanta Braves

WS Probability = 0.9970308299970712
Chicago Cubs

WS Probability = 0.9910139749698332
New York Yankees

WS Probability = 0.9650458302619158
Milwaukee Brewers

WS Probability = 0.9396427594597218
Oakland Athletics

WS Probability = 0.8784071670503562
Los Angeles Angels

WS Probability = 0.8701485224314954
Tampa Bay Rays

WS Probability = 0.8621753321905754
Minnesota Twins

WS Probability = 0.7016174136376923
Seattle Mariners

WS Probability = 0.6455688023903986
New York Mets

WS Probability = 0.5625872854145233
Colorado Rockies

WS Probability = 0.2356829356177447
St. Louis Cardinals

WS Probability = 0.1294959934781199
Cincinnati Reds

WS Probability = 0.12499468816

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


And the Astros won 2017.  Huh.  This one worked.

##### 2016 Prediction

In [22]:
# predict for 2016.
predict_the_winner(model_100, 2016, team_data, X_train_100)


WS Probability = 0.9999963160725736
Cleveland Indians

WS Probability = 0.9999829364491639
Houston Astros

WS Probability = 0.9999743320635677
Boston Red Sox

WS Probability = 0.9999163764689001
New York Yankees

WS Probability = 0.9997482501991293
Washington Nationals

WS Probability = 0.9858603063406138
Los Angeles Dodgers

WS Probability = 0.9266918870558324
Minnesota Twins

WS Probability = 0.8930945130660088
Seattle Mariners

WS Probability = 0.8324737041360648
Tampa Bay Rays

WS Probability = 0.7620155940525638
Miami Marlins

WS Probability = 0.7243618797881249
Arizona Diamondbacks

WS Probability = 0.7220477907989453
St. Louis Cardinals

WS Probability = 0.7016333178460297
Detroit Tigers

WS Probability = 0.6906966267466386
Chicago Cubs

WS Probability = 0.5847196418933889
Atlanta Braves

WS Probability = 0.5832745775024303
San Francisco Giants

WS Probability = 0.5065556013727565
Kansas City Royals

WS Probability = 0.4005409858844858
New York Mets

WS Probability = 0.19287958

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


The Cubs won 2016.  Two of three predictions were incorrect.  I want to try another model--want to improve F1 score.

What *is* interesting is that the Indians made it to the finals in 2016 and the Red Sox made it to the finals in 2018.  However, this doesn't call out the outright winner.